# Building a LLM Multi-Agent System for Certification Exam Preparation: 
# Part 1 - From Concept to Code

This notebook is a proof of concept of a LLM multi-agent system to help prepare for certification exams.

The system is design to create a list of **Flashcards** to memorize key concepts and definitions, and **Practice exams** that simulate an actual exam to test the understanding of the material and prepare for real exams.

The system is designed to be modular and flexible, so you can easily add or remove agents and tasks.

The full article that describes the system is available [here](https://numericaideas.com/blog/ai-agents-for-certification-exam-preparation/).


## Prerequisites

To run this notebook, you will need `Python 3.10+` and install the following libraries:
- python-dotenv
- crewai
- crewai-tools

1. Create a new virtual environment
    ```bash
    python -m venv .venv
    ```

2. Activate the virtual environment
    ```bash
    source .venv/bin/activate
    ```

3. Install the libraries
    
    You can install the libraries using the following command:
    ```bash
    pip install python-dotenv crewai crewai-tools
    ```

    Or using the `requirements.txt` file:
    ```bash
    pip install -r requirements.txt
    ```

4. Make sure to set the interpreter of the notebook to the Python version in the `.venv/bin` folder (see the configuration for Visual Studio Code [here](https://code.visualstudio.com/docs/python/environments)).

You are all set!

## Load Environment Variables

In [1]:
import os
from dotenv import load_dotenv
import logging

# Warning control
import warnings
warnings.filterwarnings('ignore')

#configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables from your .env file (OPENAI_API_KEY, SERPER_API_KEY, etc.)
load_dotenv() 

# Check environment variables are set by logging a fake variable TEST_ENV_VAR
logger.info(f"TEST_ENV_VAR: {os.environ['TEST_ENV_VAR']}")

INFO:__main__:TEST_ENV_VAR: environment-vars-loaded


## Load Configurations for Agents and Tasks from YAML files

In [2]:
import yaml

# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Agents, Tools, Tasks, and Crew

### Importing Libraries

In [3]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

INFO:httpx:HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


You may need to install snowflake-connector-python to import the tools if the above code fails on the second line while mentioning *snowflake-connector-python*: 

    `pip install snowflake-connector-python`

The following tools will be used:
- `SerperDevTool` to search the web for information. It requires a `SERPER_API_KEY` in the `.env` file to run properly. 

    Get your API key with 2500 free queries on [Serper website](https://serper.dev/api-key).
    
- `ScrapeWebsiteTool` to scrape information from the web.
- `WebsiteSearchTool` to navigate and research information on relevant websites.

### Set up the LLM

CrewAI uses `gpt-4o-mini` by default. Set the `OPENAI_MODEL_NAME` in the `.env` file to use a different model.

You should set the `OPENAI_API_KEY` in the `.env` file.

`OPENAI_API_KEY=your-openai-api-key`

**Note**: How to set up a custom LLM (provider, model, temperature, max_tokens, etc.) is available [here](https://docs.crewai.com/concepts/llms#setting-up-your-llm).

### Creating Agents

In [4]:
from crewai import LLM
model_provider = "openai"
model_name = "gpt-4o"
temperature = 0 # You can try different temperatures to see the impact on the crew's performance
custom_llm = LLM(
                model=f"{model_provider}/{model_name}",
                temperature=temperature,        
                api_key=os.environ['OPENAI_API_KEY']
)


In [5]:
# Uncomment the following code to use a local model
# Read my previous article to set up ollama and run local models: https://numericaideas.com/blog/build-your-own-chatgpt/
# from crewai import LLM
# model_provider = "ollama"
# model_name = "deepseek-r1:14b"
# temperature = 0.4
# custom_llm = LLM(
#                 model=f"{model_provider}/{model_name}",
#                 temperature=temperature
# )

In [6]:
lead_certification_trainer_agent = Agent(
    config=agents_config['lead_certification_trainer_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=custom_llm, # Uncomment to use a custom LLM
)

flashcard_author_agent = Agent(
    config=agents_config['flashcard_author_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=custom_llm, # Uncomment to use a custom LLM
)

exam_author_agent = Agent(
    config=agents_config['exam_author_agent'], 
    tools=[SerperDevTool(), WebsiteSearchTool()], 
    llm=custom_llm, # Uncomment to use a custom LLM
)

flashcard_quality_assurance_agent = Agent(
    config=agents_config['flashcard_quality_assurance_agent'],
    llm=custom_llm, # Uncomment to use a custom LLM
)

exam_quality_assurance_agent = Agent(
    config=agents_config['exam_quality_assurance_agent'], 
    llm=custom_llm, # Uncomment to use a custom LLM
)


INFO:alembic.runtime.migration:Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.


LLM value is already an LLM object
LLM value is already an LLM object
LLM value is already an LLM object
LLM value is already an LLM object
LLM value is already an LLM object


### Creating Tasks

In [7]:
# Get the current timestamp to name the output files for multiple experiments
from datetime import datetime
time_start = datetime.now()
timestamp_start = time_start.strftime("%Y-%m-%d %H:%M:%S")

prepare_training_resources = Task(
    config=tasks_config['prepare_training_resources'],
    agent=lead_certification_trainer_agent  
)

create_flashcards = Task(
    config=tasks_config['create_flashcards'],
    agent=flashcard_author_agent,
    context=[prepare_training_resources],
)

create_mock_exam = Task(
    config=tasks_config['create_mock_exam'],
    agent=exam_author_agent,
    context=[prepare_training_resources]
)

flashcard_file_name = f'./results/flashcards_{timestamp_start}.md'  
exam_file_name = f'./results/exam_{timestamp_start}.md'

edit_flashcards = Task(
    config=tasks_config['edit_flashcards'],
    agent=flashcard_quality_assurance_agent,
    output_file=flashcard_file_name
)

edit_mock_exam = Task(
    config=tasks_config['edit_mock_exam'],
    agent=exam_quality_assurance_agent,
    output_file=exam_file_name
)

### Creating the Crew

In [8]:
certif_preparation_crew = Crew(
    agents=[
        lead_certification_trainer_agent,
        flashcard_author_agent,
        exam_author_agent,
        flashcard_quality_assurance_agent,
        exam_quality_assurance_agent
    ],
    tasks=[
        prepare_training_resources,
        create_flashcards,
        create_mock_exam,
        edit_flashcards,
        edit_mock_exam
    ],
    verbose=True
)

### Running the Crew

In [9]:
# Inputs parameters for Google Cloud Certified Machine Learning Engineer Certification
# num_flashcards: int = 50
# num_questions: int = 50
# certification_name: str = "Google Cloud Certified Machine Learning Engineer"
# certification_link: str = "https://cloud.google.com/certification/machine-learning-engineer"

In [10]:
# Uncomment the following inputs parameters for Professional Scrum Master™ I Certification
# Customize for your certification
num_flashcards: int = 50
num_questions: int = 50
certification_name: str = "Professional Scrum Master I"
certification_link: str = "https://www.scrum.org/assessments/professional-scrum-master-i-certification"

In [11]:
result = certif_preparation_crew.kickoff(inputs={
  'certification_name': certification_name,
  'certification_link': certification_link, 
  'num_flashcards': num_flashcards,
  'num_questions': num_questions
})

01:44:08 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai


# Agent: Certification Trainer
## Task: Search and analyse the latest resources, updates and contents  related to the Professional Scrum Master I certification. Identify and collect the most relevant content that could help candidates  to get all the knowledge and skills required to pass the certification exam. Utilize the certification official website (https://www.scrum.org/assessments/professional-scrum-master-i-certification),  web search and data scraping tools to gather up-to-date information.  Focus on collecting study contents and various type of practice questions  from official and unofficial sources, and verified answers.  Include  various situational questions, a situational question being a question  that requires the candidate to apply the knowledge to a real-world scenario.



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
01:44:09 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
01:44:10 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai




# Agent: Certification Trainer
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Professional Scrum Master I certification site:scrum.org\"}"
## Tool Output: 
{'searchParameters': {'q': 'Professional Scrum Master I certification site:scrum.org', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Professional Scrum Master™ I Certification', 'link': 'https://www.scrum.org/assessments/professional-scrum-master-i-certification', 'snippet': 'The Professional Scrum Master™ I (PSM I) certification validates your knowledge of the Scrum framework, the Scrum Master accountabilities and how to apply Scrum ...', 'position': 1, 'sitelinks': [{'title': 'Professional Scrum Master', 'link': 'https://www.scrum.org/courses/professional-scrum-master-training'}, {'title': 'Applying Professional Scrum', 'link': 'https://www.scrum.org/courses/applying-professional-scrum-training'}, {'title': 'Suggested Reading', 'link': 'https://www.scrum.org/resour

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
01:44:11 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
01:44:11 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai


...



# Agent: Practice Exam Author
## Final Answer: 
**Professional Scrum Master I Mock Exam**

**Question 1:**
During a Sprint, the Development Team realizes that they will not be able to complete the work they forecasted. What should they do?
- A) Extend the Sprint duration to complete the work.
- B) Inform the Product Owner and adjust the Sprint Backlog.
- C) Cancel the Sprint and start a new one.
- D) Work overtime to complete the work.

**Correct Answer:** B) Inform the Product Owner and adjust the Sprint Backlog.

**Explanation:** The Development Team should communicate with the Product Owner to adjust the Sprint Backlog based on the new understanding of the work.

**Question 2:**
A Scrum Master is facilitating a Sprint Retrospective. The team is reluctant to speak up about issues they faced during the Sprint. What should the Scrum Master do?
- A) Skip the Retrospective and move on to the next Sprint.
- B) Encourage open communication and create a safe environment for feedback.
- C)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
01:48:45 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
01:48:48 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai




# Agent: Flashcard Editor
## Final Answer: 
**Flashcard 1:**
**Front:** During a Sprint, the Development Team realizes that they will not be able to complete the work they forecasted. What should they do?
**Back:** Inform the Product Owner and adjust the Sprint Backlog. The Development Team should communicate with the Product Owner to adjust the Sprint Backlog based on the new understanding of the work.
- [Scrum Guide - Sprint](https://www.scrum.org/resources/scrum-guide#sprint)
- [Scrum.org - Sprint Planning](https://www.scrum.org/resources/blog/sprint-planning)
- [Scrum.org - Sprint Backlog](https://www.scrum.org/resources/blog/sprint-backlog)

**Flashcard 2:**
**Front:** A Scrum Master is facilitating a Sprint Retrospective. The team is reluctant to speak up about issues they faced during the Sprint. What should the Scrum Master do?
**Back:** Encourage open communication and create a safe environment for feedback. The Scrum Master should foster an environment where team members fe

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
01:49:52 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler




# Agent: Chief Exam Officer
## Final Answer: 
# Professional Scrum Master I Mock Exam

## Instructions
This mock exam consists of 50 questions designed to help you prepare for the Professional Scrum Master I certification. Each question is followed by the correct answer and an explanation to enhance your understanding. The questions cover a range of topics aligned with the Scrum framework and the certification objectives.

## Questions

### Question 1
**During a Sprint, the Development Team realizes that they will not be able to complete the work they forecasted. What should they do?**

- A) Cancel the Sprint and start a new one.
- B) Inform the Product Owner and adjust the Sprint Backlog.
- C) Extend the Sprint duration to complete the work.
- D) Work overtime to complete the work.

**Answer:** B) Inform the Product Owner and adjust the Sprint Backlog.  
**Explanation:** The Development Team should communicate with the Product Owner to adjust the Sprint Backlog based on the new unde

## Displaying the Results
Look at the results in the markdown files in the `results` folder.

You can also look at the results in the `result` variable returned by the crew `kickoff` method. However, by default, CrewAI only returns the raw output of the last agent.

In [12]:
# Read a markdown file and display it
from IPython.display import display, Markdown

def display_markdown_file(file_name):
    with open(file_name, 'r') as file:
        markdown_content = file.read()
    display(Markdown(markdown_content))


In [13]:
# Display the flashcards
display_markdown_file(flashcard_file_name)

**Flashcard 1:**
**Front:** During a Sprint, the Development Team realizes that they will not be able to complete the work they forecasted. What should they do?
**Back:** Inform the Product Owner and adjust the Sprint Backlog. The Development Team should communicate with the Product Owner to adjust the Sprint Backlog based on the new understanding of the work.
- [Scrum Guide - Sprint](https://www.scrum.org/resources/scrum-guide#sprint)
- [Scrum.org - Sprint Planning](https://www.scrum.org/resources/blog/sprint-planning)
- [Scrum.org - Sprint Backlog](https://www.scrum.org/resources/blog/sprint-backlog)

**Flashcard 2:**
**Front:** A Scrum Master is facilitating a Sprint Retrospective. The team is reluctant to speak up about issues they faced during the Sprint. What should the Scrum Master do?
**Back:** Encourage open communication and create a safe environment for feedback. The Scrum Master should foster an environment where team members feel safe to share their thoughts and concerns.
- [Scrum Guide - Sprint Retrospective](https://www.scrum.org/resources/scrum-guide#sprint-retrospective)
- [Scrum.org - Facilitation Techniques](https://www.scrum.org/resources/blog/facilitation-techniques)
- [Scrum.org - Retrospective](https://www.scrum.org/resources/blog/retrospective)

**Flashcard 3:**
**Front:** The Product Owner is not available for the Sprint Review. What is the best course of action for the Scrum Team?
**Back:** Have the Scrum Master present the work to stakeholders. The Scrum Master can facilitate the Sprint Review in the absence of the Product Owner to ensure stakeholders are informed.
- [Scrum Guide - Sprint Review](https://www.scrum.org/resources/scrum-guide#sprint-review)
- [Scrum.org - Role of the Scrum Master](https://www.scrum.org/resources/blog/role-scrum-master)
- [Scrum.org - Stakeholder Engagement](https://www.scrum.org/resources/blog/stakeholder-engagement)

**Flashcard 4:**
**Front:** A Development Team member suggests a new tool that could improve the team's productivity. How should the Scrum Team proceed?
**Back:** The team experiments with the tool during the next Sprint. The Scrum Team can experiment with new tools and practices to improve their process and productivity.
- [Scrum Guide - Continuous Improvement](https://www.scrum.org/resources/scrum-guide#continuous-improvement)
- [Scrum.org - Experimentation in Scrum](https://www.scrum.org/resources/blog/experimentation-scrum)
- [Scrum.org - Agile Tools](https://www.scrum.org/resources/blog/agile-tools)

**Flashcard 5:**
**Front:** During a Sprint Planning meeting, the Development Team is unsure about the complexity of a Product Backlog item. What should they do?
**Back:** Ask the Product Owner for more details and clarification. The Development Team should seek clarification from the Product Owner to ensure they have a clear understanding of the item.
- [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)
- [Scrum.org - Product Backlog Refinement](https://www.scrum.org/resources/blog/product-backlog-refinement)
- [Scrum.org - Clarifying Requirements](https://www.scrum.org/resources/blog/clarifying-requirements)

**Flashcard 6:**
**Front:** The Scrum Team is consistently failing to meet the Definition of Done. What should the Scrum Master do?
**Back:** Coach the team on the importance of meeting the Definition of Done. The Scrum Master should help the team understand the importance of the Definition of Done and work towards meeting it consistently.
- [Scrum Guide - Definition of Done](https://www.scrum.org/resources/scrum-guide#definition-of-done)
- [Scrum.org - Coaching the Team](https://www.scrum.org/resources/blog/coaching-team)
- [Scrum.org - Importance of Definition of Done](https://www.scrum.org/resources/blog/importance-definition-done)

**Flashcard 7:**
**Front:** A stakeholder requests a change to the product during a Sprint. How should the Scrum Team handle this request?
**Back:** Discuss the change with the Product Owner for prioritization. The Product Owner is responsible for managing the Product Backlog and prioritizing changes based on value and impact.
- [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)
- [Scrum.org - Managing Change in Scrum](https://www.scrum.org/resources/blog/managing-change-scrum)
- [Scrum.org - Product Owner Responsibilities](https://www.scrum.org/resources/blog/product-owner-responsibilities)

**Flashcard 8:**
**Front:** The Development Team is struggling with a technical challenge that is delaying progress. What should the Scrum Master do?
**Back:** Encourage the team to collaborate and find a solution. The Scrum Master should facilitate collaboration and support the team in overcoming challenges without directly solving the problem for them.
- [Scrum Guide - Scrum Master Role](https://www.scrum.org/resources/scrum-guide#scrum-master-role)
- [Scrum.org - Facilitating Collaboration](https://www.scrum.org/resources/blog/facilitating-collaboration)
- [Scrum.org - Overcoming Challenges](https://www.scrum.org/resources/blog/overcoming-challenges)

**Flashcard 9:**
**Front:** A new team member joins the Scrum Team. How should the Scrum Master integrate them into the team?
**Back:** Facilitate their onboarding and encourage team collaboration. The Scrum Master should ensure the new team member is integrated smoothly and encourage collaboration with the rest of the team.
- [Scrum Guide - Scrum Team](https://www.scrum.org/resources/scrum-guide#scrum-team)
- [Scrum.org - Onboarding New Team Members](https://www.scrum.org/resources/blog/onboarding-new-team-members)
- [Scrum.org - Team Collaboration](https://www.scrum.org/resources/blog/team-collaboration)

**Flashcard 10:**
**Front:** The Product Owner wants to release a potentially shippable product increment at the end of each Sprint. What should the Development Team do?
**Back:** Ensure that each increment meets the Definition of Done. The Development Team should focus on delivering increments that meet the Definition of Done, ensuring they are potentially shippable.
- [Scrum Guide - Increment](https://www.scrum.org/resources/scrum-guide#increment)
- [Scrum.org - Definition of Done](https://www.scrum.org/resources/blog/definition-done)
- [Scrum.org - Releasing Increments](https://www.scrum.org/resources/blog/releasing-increments)

**Flashcard 11:**
**Front:** A Scrum Team is working on a complex project with high uncertainty. How should they approach Sprint Planning?
**Back:** Focus on short-term goals and adapt as new information emerges. In complex projects, the Scrum Team should focus on short-term goals and adapt their plans as they gain more information.
- [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)
- [Scrum.org - Managing Complexity](https://www.scrum.org/resources/blog/managing-complexity)
- [Scrum.org - Adapting Plans](https://www.scrum.org/resources/blog/adapting-plans)

**Flashcard 12:**
**Front:** The Scrum Master notices that the Daily Scrum is taking longer than 15 minutes. What should they do?
**Back:** Facilitate the meeting to ensure it stays focused and concise. The Scrum Master should help the team keep the Daily Scrum focused and within the 15-minute timebox.
- [Scrum Guide - Daily Scrum](https://www.scrum.org/resources/scrum-guide#daily-scrum)
- [Scrum.org - Timeboxing in Scrum](https://www.scrum.org/resources/blog/timeboxing-scrum)
- [Scrum.org - Effective Daily Scrum](https://www.scrum.org/resources/blog/effective-daily-scrum)

**Flashcard 13:**
**Front:** A Scrum Team is distributed across different time zones. How should they conduct the Daily Scrum?
**Back:** Use asynchronous communication tools to accommodate time differences. Asynchronous communication tools can help distributed teams conduct effective Daily Scrums despite time zone differences.
- [Scrum Guide - Daily Scrum](https://www.scrum.org/resources/scrum-guide#daily-scrum)
- [Scrum.org - Distributed Teams](https://www.scrum.org/resources/blog/distributed-teams)
- [Scrum.org - Asynchronous Communication](https://www.scrum.org/resources/blog/asynchronous-communication)

**Flashcard 14:**
**Front:** The Product Owner is frequently changing the priorities of the Product Backlog. How should the Scrum Team respond?
**Back:** Discuss the impact of changes with the Product Owner and team. The Scrum Team should collaborate with the Product Owner to understand the impact of changes and adjust their plans accordingly.
- [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)
- [Scrum.org - Managing Product Backlog](https://www.scrum.org/resources/blog/managing-product-backlog)
- [Scrum.org - Prioritization in Scrum](https://www.scrum.org/resources/blog/prioritization-scrum)

**Flashcard 15:**
**Front:** A Scrum Team is consistently delivering increments that do not meet stakeholder expectations. What should the Scrum Master do?
**Back:** Facilitate better communication between the team and stakeholders. The Scrum Master should ensure effective communication between the team and stakeholders to align expectations and deliver value.
- [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)
- [Scrum.org - Communication in Scrum](https://www.scrum.org/resources/blog/communication-scrum)
- [Scrum.org - Aligning Expectations](https://www.scrum.org/resources/blog/aligning-expectations)

**Flashcard 16:**
**Front:** The Development Team is unsure about the acceptance criteria for a Product Backlog item. What should they do?
**Back:** Seek clarification from the Product Owner. The Development Team should clarify acceptance criteria with the Product Owner to ensure they meet the desired outcome.
- [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)
- [Scrum.org - Acceptance Criteria](https://www.scrum.org/resources/blog/acceptance-criteria)
- [Scrum.org - Clarifying Requirements](https://www.scrum.org/resources/blog/clarifying-requirements)

**Flashcard 17:**
**Front:** A Scrum Team is working on a project with rapidly changing requirements. How should they manage the Product Backlog?
**Back:** Continuously refine and prioritize the Product Backlog. The Scrum Team should continuously refine and prioritize the Product Backlog to adapt to changing requirements.
- [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)
- [Scrum.org - Product Backlog Refinement](https://www.scrum.org/resources/blog/product-backlog-refinement)
- [Scrum.org - Managing Change in Scrum](https://www.scrum.org/resources/blog/managing-change-scrum)

**Flashcard 18:**
**Front:** The Scrum Master observes that the team is not collaborating effectively. What should they do?
**Back:** Facilitate team-building activities to enhance collaboration. The Scrum Master should foster collaboration through team-building activities and support the team in working together effectively.
- [Scrum Guide - Scrum Team](https://www.scrum.org/resources/scrum-guide#scrum-team)
- [Scrum.org - Team Building](https://www.scrum.org/resources/blog/team-building)
- [Scrum.org - Enhancing Collaboration](https://www.scrum.org/resources/blog/enhancing-collaboration)

**Flashcard 19:**
**Front:** A stakeholder requests a detailed project plan with fixed deadlines. How should the Scrum Team respond?
**Back:** Explain the iterative nature of Scrum and focus on delivering value. The Scrum Team should communicate the benefits of the iterative approach and focus on delivering value incrementally.
- [Scrum Guide - Scrum Framework](https://www.scrum.org/resources/scrum-guide#scrum-framework)
- [Scrum.org - Iterative Development](https://www.scrum.org/resources/blog/iterative-development)
- [Scrum.org - Delivering Value](https://www.scrum.org/resources/blog/delivering-value)

**Flashcard 20:**
**Front:** The Development Team is consistently completing work ahead of schedule. What should the Scrum Master do?
**Back:** Investigate if the work is meeting the Definition of Done. The Scrum Master should ensure that the work meets the Definition of Done and is of high quality before taking on more work.
- [Scrum Guide - Definition of Done](https://www.scrum.org/resources/scrum-guide#definition-of-done)
- [Scrum.org - Quality in Scrum](https://www.scrum.org/resources/blog/quality-scrum)
- [Scrum.org - Definition of Done](https://www.scrum.org/resources/blog/definition-done)

**Flashcard 21:**
**Front:** A Scrum Team is struggling to deliver increments due to technical debt. What should they do?
**Back:** Allocate time in each Sprint to address technical debt. The Scrum Team should allocate time to address technical debt to maintain the quality and sustainability of the product.
- [Scrum Guide - Increment](https://www.scrum.org/resources/scrum-guide#increment)
- [Scrum.org - Technical Debt](https://www.scrum.org/resources/blog/technical-debt)
- [Scrum.org - Managing Technical Debt](https://www.scrum.org/resources/blog/managing-technical-debt)

**Flashcard 22:**
**Front:** The Product Owner is not actively participating in Sprint Planning. What should the Scrum Master do?
**Back:** Encourage the Product Owner to engage and provide input. The Scrum Master should ensure the Product Owner is actively involved in Sprint Planning to provide valuable input.
- [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)
- [Scrum.org - Product Owner Engagement](https://www.scrum.org/resources/blog/product-owner-engagement)
- [Scrum.org - Role of the Product Owner](https://www.scrum.org/resources/blog/role-product-owner)

**Flashcard 23:**
**Front:** A Scrum Team is working on a project with multiple Scrum Teams. How should they coordinate their work?
**Back:** Use a shared Product Backlog and coordinate through Scrum of Scrums. Coordination through a shared Product Backlog and Scrum of Scrums helps multiple teams work together effectively.
- [Scrum Guide - Scrum of Scrums](https://www.scrum.org/resources/scrum-guide#scrum-of-scrums)
- [Scrum.org - Multi-Team Coordination](https://www.scrum.org/resources/blog/multi-team-coordination)
- [Scrum.org - Shared Product Backlog](https://www.scrum.org/resources/blog/shared-product-backlog)

**Flashcard 24:**
**Front:** The Development Team is consistently underestimating the effort required for Product Backlog items. What should they do?
**Back:** Review past estimates and adjust their estimation process. The Development Team should learn from past estimations and improve their estimation process for future items.
- [Scrum Guide - Estimation](https://www.scrum.org/resources/scrum-guide#estimation)
- [Scrum.org - Improving Estimation](https://www.scrum.org/resources/blog/improving-estimation)
- [Scrum.org - Estimation Techniques](https://www.scrum.org/resources/blog/estimation-techniques)

**Flashcard 25:**
**Front:** A Scrum Team is facing resistance from stakeholders who are unfamiliar with Scrum. What should the Scrum Master do?
**Back:** Educate stakeholders about Scrum and its benefits. The Scrum Master should educate stakeholders about Scrum to gain their support and understanding.
- [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)
- [Scrum.org - Educating Stakeholders](https://www.scrum.org/resources/blog/educating-stakeholders)
- [Scrum.org - Benefits of Scrum](https://www.scrum.org/resources/blog/benefits-scrum)

**Flashcard 26:**
**Front:** The Product Owner wants to add a new feature to the current Sprint. How should the Scrum Team handle this request?
**Back:** Discuss the request with the Development Team and adjust the Sprint Backlog if necessary. The Scrum Team should collaborate to assess the impact of the new feature and adjust the Sprint Backlog if needed.
- [Scrum Guide - Sprint Backlog](https://www.scrum.org/resources/scrum-guide#sprint-backlog)
- [Scrum.org - Managing Sprint Backlog](https://www.scrum.org/resources/blog/managing-sprint-backlog)
- [Scrum.org - Handling Change Requests](https://www.scrum.org/resources/blog/handling-change-requests)

**Flashcard 27:**
**Front:** A Scrum Team is working on a project with a fixed deadline. How should they manage their work to meet the deadline?
**Back:** Focus on delivering the highest priority items first. The Scrum Team should prioritize high-value items to ensure they deliver the most important features by the deadline.
- [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)
- [Scrum.org - Prioritization Techniques](https://www.scrum.org/resources/blog/prioritization-techniques)
- [Scrum.org - Meeting Deadlines](https://www.scrum.org/resources/blog/meeting-deadlines)

**Flashcard 28:**
**Front:** The Development Team is struggling to complete work due to frequent interruptions. What should the Scrum Master do?
**Back:** Shield the team from interruptions and facilitate focused work. The Scrum Master should protect the team from interruptions to enable them to focus on their work and deliver value.
- [Scrum Guide - Scrum Master Role](https://www.scrum.org/resources/scrum-guide#scrum-master-role)
- [Scrum.org - Managing Interruptions](https://www.scrum.org/resources/blog/managing-interruptions)
- [Scrum.org - Focused Work](https://www.scrum.org/resources/blog/focused-work)

**Flashcard 29:**
**Front:** A Scrum Team is working on a project with high technical complexity. How should they approach Sprint Planning?
**Back:** Focus on short-term goals and adapt as they learn more. In complex projects, the Scrum Team should focus on short-term goals and adapt their plans as they gain more information.
- [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)
- [Scrum.org - Managing Complexity](https://www.scrum.org/resources/blog/managing-complexity)
- [Scrum.org - Adapting Plans](https://www.scrum.org/resources/blog/adapting-plans)

**Flashcard 30:**
**Front:** The Product Owner is frequently unavailable for important meetings. What should the Scrum Master do?
**Back:** Encourage the Product Owner to prioritize their availability. The Scrum Master should ensure the Product Owner is available for important meetings to provide valuable input.
- [Scrum Guide - Product Owner Role](https://www.scrum.org/resources/scrum-guide#product-owner-role)
- [Scrum.org - Importance of Product Owner](https://www.scrum.org/resources/blog/importance-product-owner)
- [Scrum.org - Engaging Product Owners](https://www.scrum.org/resources/blog/engaging-product-owners)

**Flashcard 31:**
**Front:** A Scrum Team is working on a project with rapidly changing requirements. How should they manage the Product Backlog?
**Back:** Continuously refine and prioritize the Product Backlog. The Scrum Team should continuously refine and prioritize the Product Backlog to adapt to changing requirements.
- [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)
- [Scrum.org - Product Backlog Refinement](https://www.scrum.org/resources/blog/product-backlog-refinement)
- [Scrum.org - Managing Change in Scrum](https://www.scrum.org/resources/blog/managing-change-scrum)

**Flashcard 32:**
**Front:** The Development Team is unsure about the acceptance criteria for a Product Backlog item. What should they do?
**Back:** Seek clarification from the Product Owner. The Development Team should clarify acceptance criteria with the Product Owner to ensure they meet the desired outcome.
- [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)
- [Scrum.org - Acceptance Criteria](https://www.scrum.org/resources/blog/acceptance-criteria)
- [Scrum.org - Clarifying Requirements](https://www.scrum.org/resources/blog/clarifying-requirements)

**Flashcard 33:**
**Front:** A Scrum Team is consistently delivering increments that do not meet stakeholder expectations. What should the Scrum Master do?
**Back:** Facilitate better communication between the team and stakeholders. The Scrum Master should ensure effective communication between the team and stakeholders to align expectations and deliver value.
- [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)
- [Scrum.org - Communication in Scrum](https://www.scrum.org/resources/blog/communication-scrum)
- [Scrum.org - Aligning Expectations](https://www.scrum.org/resources/blog/aligning-expectations)

**Flashcard 34:**
**Front:** The Product Owner is not actively participating in Sprint Planning. What should the Scrum Master do?
**Back:** Encourage the Product Owner to engage and provide input. The Scrum Master should ensure the Product Owner is actively involved in Sprint Planning to provide valuable input.
- [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)
- [Scrum.org - Product Owner Engagement](https://www.scrum.org/resources/blog/product-owner-engagement)
- [Scrum.org - Role of the Product Owner](https://www.scrum.org/resources/blog/role-product-owner)

**Flashcard 35:**
**Front:** A Scrum Team is working on a project with multiple Scrum Teams. How should they coordinate their work?
**Back:** Use a shared Product Backlog and coordinate through Scrum of Scrums. Coordination through a shared Product Backlog and Scrum of Scrums helps multiple teams work together effectively.
- [Scrum Guide - Scrum of Scrums](https://www.scrum.org/resources/scrum-guide#scrum-of-scrums)
- [Scrum.org - Multi-Team Coordination](https://www.scrum.org/resources/blog/multi-team-coordination)
- [Scrum.org - Shared Product Backlog](https://www.scrum.org/resources/blog/shared-product-backlog)

**Flashcard 36:**
**Front:** The Development Team is consistently underestimating the effort required for Product Backlog items. What should they do?
**Back:** Review past estimates and adjust their estimation process. The Development Team should learn from past estimations and improve their estimation process for future items.
- [Scrum Guide - Estimation](https://www.scrum.org/resources/scrum-guide#estimation)
- [Scrum.org - Improving Estimation](https://www.scrum.org/resources/blog/improving-estimation)
- [Scrum.org - Estimation Techniques](https://www.scrum.org/resources/blog/estimation-techniques)

**Flashcard 37:**
**Front:** A Scrum Team is facing resistance from stakeholders who are unfamiliar with Scrum. What should the Scrum Master do?
**Back:** Educate stakeholders about Scrum and its benefits. The Scrum Master should educate stakeholders about Scrum to gain their support and understanding.
- [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)
- [Scrum.org - Educating Stakeholders](https://www.scrum.org/resources/blog/educating-stakeholders)
- [Scrum.org - Benefits of Scrum](https://www.scrum.org/resources/blog/benefits-scrum)

**Flashcard 38:**
**Front:** The Product Owner wants to add a new feature to the current Sprint. How should the Scrum Team handle this request?
**Back:** Discuss the request with the Development Team and adjust the Sprint Backlog if necessary. The Scrum Team should collaborate to assess the impact of the new feature and adjust the Sprint Backlog if needed.
- [Scrum Guide - Sprint Backlog](https://www.scrum.org/resources/scrum-guide#sprint-backlog)
- [Scrum.org - Managing Sprint Backlog](https://www.scrum.org/resources/blog/managing-sprint-backlog)
- [Scrum.org - Handling Change Requests](https://www.scrum.org/resources/blog/handling-change-requests)

**Flashcard 39:**
**Front:** A Scrum Team is working on a project with a fixed deadline. How should they manage their work to meet the deadline?
**Back:** Focus on delivering the highest priority items first. The Scrum Team should prioritize high-value items to ensure they deliver the most important features by the deadline.
- [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)
- [Scrum.org - Prioritization Techniques](https://www.scrum.org/resources/blog/prioritization-techniques)
- [Scrum.org - Meeting Deadlines](https://www.scrum.org/resources/blog/meeting-deadlines)

**Flashcard 40:**
**Front:** The Development Team is struggling to complete work due to frequent interruptions. What should the Scrum Master do?
**Back:** Shield the team from interruptions and facilitate focused work. The Scrum Master should protect the team from interruptions to enable them to focus on their work and deliver value.
- [Scrum Guide - Scrum Master Role](https://www.scrum.org/resources/scrum-guide#scrum-master-role)
- [Scrum.org - Managing Interruptions](https://www.scrum.org/resources/blog/managing-interruptions)
- [Scrum.org - Focused Work](https://www.scrum.org/resources/blog/focused-work)

**Flashcard 41:**
**Front:** A Scrum Team is working on a project with high technical complexity. How should they approach Sprint Planning?
**Back:** Focus on short-term goals and adapt as they learn more. In complex projects, the Scrum Team should focus on short-term goals and adapt their plans as they gain more information.
- [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)
- [Scrum.org - Managing Complexity](https://www.scrum.org/resources/blog/managing-complexity)
- [Scrum.org - Adapting Plans](https://www.scrum.org/resources/blog/adapting-plans)

**Flashcard 42:**
**Front:** The Product Owner is frequently unavailable for important meetings. What should the Scrum Master do?
**Back:** Encourage the Product Owner to prioritize their availability. The Scrum Master should ensure the Product Owner is available for important meetings to provide valuable input.
- [Scrum Guide - Product Owner Role](https://www.scrum.org/resources/scrum-guide#product-owner-role)
- [Scrum.org - Importance of Product Owner](https://www.scrum.org/resources/blog/importance-product-owner)
- [Scrum.org - Engaging Product Owners](https://www.scrum.org/resources/blog/engaging-product-owners)

**Flashcard 43:**
**Front:** A Scrum Team is working on a project with rapidly changing requirements. How should they manage the Product Backlog?
**Back:** Continuously refine and prioritize the Product Backlog. The Scrum Team should continuously refine and prioritize the Product Backlog to adapt to changing requirements.
- [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)
- [Scrum.org - Product Backlog Refinement](https://www.scrum.org/resources/blog/product-backlog-refinement)
- [Scrum.org - Managing Change in Scrum](https://www.scrum.org/resources/blog/managing-change-scrum)

**Flashcard 44:**
**Front:** The Development Team is unsure about the acceptance criteria for a Product Backlog item. What should they do?
**Back:** Seek clarification from the Product Owner. The Development Team should clarify acceptance criteria with the Product Owner to ensure they meet the desired outcome.
- [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)
- [Scrum.org - Acceptance Criteria](https://www.scrum.org/resources/blog/acceptance-criteria)
- [Scrum.org - Clarifying Requirements](https://www.scrum.org/resources/blog/clarifying-requirements)

**Flashcard 45:**
**Front:** A Scrum Team is consistently delivering increments that do not meet stakeholder expectations. What should the Scrum Master do?
**Back:** Facilitate better communication between the team and stakeholders. The Scrum Master should ensure effective communication between the team and stakeholders to align expectations and deliver value.
- [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)
- [Scrum.org - Communication in Scrum](https://www.scrum.org/resources/blog/communication-scrum)
- [Scrum.org - Aligning Expectations](https://www.scrum.org/resources/blog/aligning-expectations)

**Flashcard 46:**
**Front:** The Product Owner is not actively participating in Sprint Planning. What should the Scrum Master do?
**Back:** Encourage the Product Owner to engage and provide input. The Scrum Master should ensure the Product Owner is actively involved in Sprint Planning to provide valuable input.
- [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)
- [Scrum.org - Product Owner Engagement](https://www.scrum.org/resources/blog/product-owner-engagement)
- [Scrum.org - Role of the Product Owner](https://www.scrum.org/resources/blog/role-product-owner)

**Flashcard 47:**
**Front:** A Scrum Team is working on a project with multiple Scrum Teams. How should they coordinate their work?
**Back:** Use a shared Product Backlog and coordinate through Scrum of Scrums. Coordination through a shared Product Backlog and Scrum of Scrums helps multiple teams work together effectively.
- [Scrum Guide - Scrum of Scrums](https://www.scrum.org/resources/scrum-guide#scrum-of-scrums)
- [Scrum.org - Multi-Team Coordination](https://www.scrum.org/resources/blog/multi-team-coordination)
- [Scrum.org - Shared Product Backlog](https://www.scrum.org/resources/blog/shared-product-backlog)

**Flashcard 48:**
**Front:** The Development Team is consistently underestimating the effort required for Product Backlog items. What should they do?
**Back:** Review past estimates and adjust their estimation process. The Development Team should learn from past estimations and improve their estimation process for future items.
- [Scrum Guide - Estimation](https://www.scrum.org/resources/scrum-guide#estimation)
- [Scrum.org - Improving Estimation](https://www.scrum.org/resources/blog/improving-estimation)
- [Scrum.org - Estimation Techniques](https://www.scrum.org/resources/blog/estimation-techniques)

**Flashcard 49:**
**Front:** A Scrum Team is facing resistance from stakeholders who are unfamiliar with Scrum. What should the Scrum Master do?
**Back:** Educate stakeholders about Scrum and its benefits. The Scrum Master should educate stakeholders about Scrum to gain their support and understanding.
- [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)
- [Scrum.org - Educating Stakeholders](https://www.scrum.org/resources/blog/educating-stakeholders)
- [Scrum.org - Benefits of Scrum](https://www.scrum.org/resources/blog/benefits-scrum)

**Flashcard 50:**
**Front:** The Product Owner wants to add a new feature to the current Sprint. How should the Scrum Team handle this request?
**Back:** Discuss the request with the Development Team and adjust the Sprint Backlog if necessary. The Scrum Team should collaborate to assess the impact of the new feature and adjust the Sprint Backlog if needed.
- [Scrum Guide - Sprint Backlog](https://www.scrum.org/resources/scrum-guide#sprint-backlog)
- [Scrum.org - Managing Sprint Backlog](https://www.scrum.org/resources/blog/managing-sprint-backlog)
- [Scrum.org - Handling Change Requests](https://www.scrum.org/resources/blog/handling-change-requests)

In [14]:

# Display the exam
display_markdown_file(exam_file_name)

# Professional Scrum Master I Mock Exam

## Instructions
This mock exam consists of 50 questions designed to help you prepare for the Professional Scrum Master I certification. Each question is followed by the correct answer and an explanation to enhance your understanding. The questions cover a range of topics aligned with the Scrum framework and the certification objectives.

## Questions

### Question 1
**During a Sprint, the Development Team realizes that they will not be able to complete the work they forecasted. What should they do?**

- A) Cancel the Sprint and start a new one.
- B) Inform the Product Owner and adjust the Sprint Backlog.
- C) Extend the Sprint duration to complete the work.
- D) Work overtime to complete the work.

**Answer:** B) Inform the Product Owner and adjust the Sprint Backlog.  
**Explanation:** The Development Team should communicate with the Product Owner to adjust the Sprint Backlog based on the new understanding of the work. [Scrum Guide - Sprint](https://www.scrum.org/resources/scrum-guide#sprint)

### Question 2
**A Scrum Master is facilitating a Sprint Retrospective. The team is reluctant to speak up about issues they faced during the Sprint. What should the Scrum Master do?**

- A) Skip the retrospective and move on to the next Sprint.
- B) Encourage open communication and create a safe environment for feedback.
- C) Force team members to speak up.
- D) Document the issues without team input.

**Answer:** B) Encourage open communication and create a safe environment for feedback.  
**Explanation:** The Scrum Master should foster an environment where team members feel safe to share their thoughts and concerns. [Scrum Guide - Sprint Retrospective](https://www.scrum.org/resources/scrum-guide#sprint-retrospective)

### Question 3
**The Product Owner is not available for the Sprint Review. What is the best course of action for the Scrum Team?**

- A) Cancel the Sprint Review.
- B) Have the Scrum Master present the work to stakeholders.
- C) Postpone the Sprint Review until the Product Owner is available.
- D) Conduct the Sprint Review without the Product Owner.

**Answer:** B) Have the Scrum Master present the work to stakeholders.  
**Explanation:** The Scrum Master can facilitate the Sprint Review in the absence of the Product Owner to ensure stakeholders are informed. [Scrum Guide - Sprint Review](https://www.scrum.org/resources/scrum-guide#sprint-review)

### Question 4
**A Development Team member suggests a new tool that could improve the team's productivity. How should the Scrum Team proceed?**

- A) Ignore the suggestion and continue with current tools.
- B) The team experiments with the tool during the next Sprint.
- C) Implement the tool immediately without testing.
- D) Wait for management approval before experimenting.

**Answer:** B) The team experiments with the tool during the next Sprint.  
**Explanation:** The Scrum Team can experiment with new tools and practices to improve their process and productivity. [Scrum Guide - Continuous Improvement](https://www.scrum.org/resources/scrum-guide#continuous-improvement)

### Question 5
**During a Sprint Planning meeting, the Development Team is unsure about the complexity of a Product Backlog item. What should they do?**

- A) Estimate the item based on assumptions.
- B) Ask the Product Owner for more details and clarification.
- C) Skip the item and move to the next one.
- D) Assign a high complexity to be safe.

**Answer:** B) Ask the Product Owner for more details and clarification.  
**Explanation:** The Development Team should seek clarification from the Product Owner to ensure they have a clear understanding of the item. [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)

### Question 6
**The Scrum Team is consistently failing to meet the Definition of Done. What should the Scrum Master do?**

- A) Lower the Definition of Done to match the team's capabilities.
- B) Coach the team on the importance of meeting the Definition of Done.
- C) Ignore the issue and focus on other priorities.
- D) Extend the Sprint duration to meet the Definition of Done.

**Answer:** B) Coach the team on the importance of meeting the Definition of Done.  
**Explanation:** The Scrum Master should help the team understand the importance of the Definition of Done and work towards meeting it consistently. [Scrum Guide - Definition of Done](https://www.scrum.org/resources/scrum-guide#definition-of-done)

### Question 7
**A stakeholder requests a change to the product during a Sprint. How should the Scrum Team handle this request?**

- A) Implement the change immediately.
- B) Discuss the change with the Product Owner for prioritization.
- C) Reject the change and continue with the current plan.
- D) Add the change to the next Sprint without discussion.

**Answer:** B) Discuss the change with the Product Owner for prioritization.  
**Explanation:** The Product Owner is responsible for managing the Product Backlog and prioritizing changes based on value and impact. [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)

### Question 8
**The Development Team is struggling with a technical challenge that is delaying progress. What should the Scrum Master do?**

- A) Solve the problem for the team.
- B) Encourage the team to collaborate and find a solution.
- C) Ignore the issue and let the team handle it.
- D) Assign the problem to an external expert.

**Answer:** B) Encourage the team to collaborate and find a solution.  
**Explanation:** The Scrum Master should facilitate collaboration and support the team in overcoming challenges without directly solving the problem for them. [Scrum Guide - Scrum Master Role](https://www.scrum.org/resources/scrum-guide#scrum-master-role)

### Question 9
**A new team member joins the Scrum Team. How should the Scrum Master integrate them into the team?**

- A) Leave the new member to figure things out on their own.
- B) Facilitate their onboarding and encourage team collaboration.
- C) Assign them a mentor from outside the team.
- D) Delay their involvement until the next Sprint.

**Answer:** B) Facilitate their onboarding and encourage team collaboration.  
**Explanation:** The Scrum Master should ensure the new team member is integrated smoothly and encourage collaboration with the rest of the team. [Scrum Guide - Scrum Team](https://www.scrum.org/resources/scrum-guide#scrum-team)

### Question 10
**The Product Owner wants to release a potentially shippable product increment at the end of each Sprint. What should the Development Team do?**

- A) Release only when the entire product is complete.
- B) Ensure that each increment meets the Definition of Done.
- C) Focus on speed over quality to meet the deadline.
- D) Release increments only when stakeholders request.

**Answer:** B) Ensure that each increment meets the Definition of Done.  
**Explanation:** The Development Team should focus on delivering increments that meet the Definition of Done, ensuring they are potentially shippable. [Scrum Guide - Increment](https://www.scrum.org/resources/scrum-guide#increment)

### Question 11
**A Scrum Team is working on a complex project with high uncertainty. How should they approach Sprint Planning?**

- A) Plan the entire project in detail upfront.
- B) Focus on short-term goals and adapt as new information emerges.
- C) Avoid planning and work on tasks as they come.
- D) Set fixed goals for the entire project duration.

**Answer:** B) Focus on short-term goals and adapt as new information emerges.  
**Explanation:** In complex projects, the Scrum Team should focus on short-term goals and adapt their plans as they gain more information. [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)

### Question 12
**The Scrum Master notices that the Daily Scrum is taking longer than 15 minutes. What should they do?**

- A) Allow the meeting to continue as long as needed.
- B) Facilitate the meeting to ensure it stays focused and concise.
- C) Cancel the Daily Scrum and reschedule it.
- D) Limit the number of participants to reduce time.

**Answer:** B) Facilitate the meeting to ensure it stays focused and concise.  
**Explanation:** The Scrum Master should help the team keep the Daily Scrum focused and within the 15-minute timebox. [Scrum Guide - Daily Scrum](https://www.scrum.org/resources/scrum-guide#daily-scrum)

### Question 13
**A Scrum Team is distributed across different time zones. How should they conduct the Daily Scrum?**

- A) Conduct the Daily Scrum at a time convenient for the Scrum Master.
- B) Use asynchronous communication tools to accommodate time differences.
- C) Skip the Daily Scrum due to time zone challenges.
- D) Conduct multiple Daily Scrums for different time zones.

**Answer:** B) Use asynchronous communication tools to accommodate time differences.  
**Explanation:** Asynchronous communication tools can help distributed teams conduct effective Daily Scrums despite time zone differences. [Scrum Guide - Daily Scrum](https://www.scrum.org/resources/scrum-guide#daily-scrum)

### Question 14
**The Product Owner is frequently changing the priorities of the Product Backlog. How should the Scrum Team respond?**

- A) Ignore the changes and stick to the original plan.
- B) Discuss the impact of changes with the Product Owner and team.
- C) Implement changes without discussion.
- D) Request a fixed Product Backlog for the entire project.

**Answer:** B) Discuss the impact of changes with the Product Owner and team.  
**Explanation:** The Scrum Team should collaborate with the Product Owner to understand the impact of changes and adjust their plans accordingly. [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)

### Question 15
**A Scrum Team is consistently delivering increments that do not meet stakeholder expectations. What should the Scrum Master do?**

- A) Ignore stakeholder feedback and focus on the team's goals.
- B) Facilitate better communication between the team and stakeholders.
- C) Replace team members to improve performance.
- D) Extend the Sprint duration to meet expectations.

**Answer:** B) Facilitate better communication between the team and stakeholders.  
**Explanation:** The Scrum Master should ensure effective communication between the team and stakeholders to align expectations and deliver value. [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)

### Question 16
**The Development Team is unsure about the acceptance criteria for a Product Backlog item. What should they do?**

- A) Make assumptions and proceed with the work.
- B) Seek clarification from the Product Owner.
- C) Skip the item and move to the next one.
- D) Use previous acceptance criteria as a reference.

**Answer:** B) Seek clarification from the Product Owner.  
**Explanation:** The Development Team should clarify acceptance criteria with the Product Owner to ensure they meet the desired outcome. [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)

### Question 17
**A Scrum Team is working on a project with rapidly changing requirements. How should they manage the Product Backlog?**

- A) Freeze the Product Backlog to prevent changes.
- B) Continuously refine and prioritize the Product Backlog.
- C) Ignore changes and focus on the original plan.
- D) Request a fixed scope from stakeholders.

**Answer:** B) Continuously refine and prioritize the Product Backlog.  
**Explanation:** The Scrum Team should continuously refine and prioritize the Product Backlog to adapt to changing requirements. [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)

### Question 18
**The Scrum Master observes that the team is not collaborating effectively. What should they do?**

- A) Assign tasks to team members to force collaboration.
- B) Facilitate team-building activities to enhance collaboration.
- C) Ignore the issue and let the team resolve it.
- D) Replace team members to improve collaboration.

**Answer:** B) Facilitate team-building activities to enhance collaboration.  
**Explanation:** The Scrum Master should foster collaboration through team-building activities and support the team in working together effectively. [Scrum Guide - Scrum Team](https://www.scrum.org/resources/scrum-guide#scrum-team)

### Question 19
**A stakeholder requests a detailed project plan with fixed deadlines. How should the Scrum Team respond?**

- A) Provide a detailed plan with fixed deadlines.
- B) Explain the iterative nature of Scrum and focus on delivering value.
- C) Ignore the request and continue with the current process.
- D) Commit to fixed deadlines to satisfy the stakeholder.

**Answer:** B) Explain the iterative nature of Scrum and focus on delivering value.  
**Explanation:** The Scrum Team should communicate the benefits of the iterative approach and focus on delivering value incrementally. [Scrum Guide - Scrum Framework](https://www.scrum.org/resources/scrum-guide#scrum-framework)

### Question 20
**The Development Team is consistently completing work ahead of schedule. What should the Scrum Master do?**

- A) Increase the workload for the team.
- B) Investigate if the work is meeting the Definition of Done.
- C) Shorten the Sprint duration to match the team's pace.
- D) Ignore the issue and continue as is.

**Answer:** B) Investigate if the work is meeting the Definition of Done.  
**Explanation:** The Scrum Master should ensure that the work meets the Definition of Done and is of high quality before taking on more work. [Scrum Guide - Definition of Done](https://www.scrum.org/resources/scrum-guide#definition-of-done)

### Question 21
**A Scrum Team is struggling to deliver increments due to technical debt. What should they do?**

- A) Ignore technical debt and focus on new features.
- B) Allocate time in each Sprint to address technical debt.
- C) Request additional resources to handle technical debt.
- D) Delay addressing technical debt until the project is complete.

**Answer:** B) Allocate time in each Sprint to address technical debt.  
**Explanation:** The Scrum Team should allocate time to address technical debt to maintain the quality and sustainability of the product. [Scrum Guide - Increment](https://www.scrum.org/resources/scrum-guide#increment)

### Question 22
**The Product Owner is not actively participating in Sprint Planning. What should the Scrum Master do?**

- A) Conduct Sprint Planning without the Product Owner.
- B) Encourage the Product Owner to engage and provide input.
- C) Assign the Product Owner's responsibilities to the Scrum Master.
- D) Skip Sprint Planning and start the Sprint.

**Answer:** B) Encourage the Product Owner to engage and provide input.  
**Explanation:** The Scrum Master should ensure the Product Owner is actively involved in Sprint Planning to provide valuable input. [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)

### Question 23
**A Scrum Team is working on a project with multiple Scrum Teams. How should they coordinate their work?**

- A) Work independently and avoid interference.
- B) Use a shared Product Backlog and coordinate through Scrum of Scrums.
- C) Assign a project manager to oversee coordination.
- D) Merge all teams into one large team.

**Answer:** B) Use a shared Product Backlog and coordinate through Scrum of Scrums.  
**Explanation:** Coordination through a shared Product Backlog and Scrum of Scrums helps multiple teams work together effectively. [Scrum Guide - Scrum of Scrums](https://www.scrum.org/resources/scrum-guide#scrum-of-scrums)

### Question 24
**The Development Team is consistently underestimating the effort required for Product Backlog items. What should they do?**

- A) Increase all estimates by a fixed percentage.
- B) Review past estimates and adjust their estimation process.
- C) Ignore estimation errors and continue as is.
- D) Use a different estimation technique for each item.

**Answer:** B) Review past estimates and adjust their estimation process.  
**Explanation:** The Development Team should learn from past estimations and improve their estimation process for future items. [Scrum Guide - Estimation](https://www.scrum.org/resources/scrum-guide#estimation)

### Question 25
**A Scrum Team is facing resistance from stakeholders who are unfamiliar with Scrum. What should the Scrum Master do?**

- A) Ignore stakeholder concerns and focus on the team's work.
- B) Educate stakeholders about Scrum and its benefits.
- C) Replace stakeholders with those familiar with Scrum.
- D) Adjust the Scrum process to meet stakeholder demands.

**Answer:** B) Educate stakeholders about Scrum and its benefits.  
**Explanation:** The Scrum Master should educate stakeholders about Scrum to gain their support and understanding. [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)

### Question 26
**The Product Owner wants to add a new feature to the current Sprint. How should the Scrum Team handle this request?**

- A) Implement the feature immediately.
- B) Discuss the request with the Development Team and adjust the Sprint Backlog if necessary.
- C) Reject the request to maintain focus.
- D) Add the feature to the next Sprint without discussion.

**Answer:** B) Discuss the request with the Development Team and adjust the Sprint Backlog if necessary.  
**Explanation:** The Scrum Team should collaborate to assess the impact of the new feature and adjust the Sprint Backlog if needed. [Scrum Guide - Sprint Backlog](https://www.scrum.org/resources/scrum-guide#sprint-backlog)

### Question 27
**A Scrum Team is working on a project with a fixed deadline. How should they manage their work to meet the deadline?**

- A) Work overtime to meet the deadline.
- B) Focus on delivering the highest priority items first.
- C) Extend the deadline to ensure quality.
- D) Reduce the scope of the project.

**Answer:** B) Focus on delivering the highest priority items first.  
**Explanation:** The Scrum Team should prioritize high-value items to ensure they deliver the most important features by the deadline. [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)

### Question 28
**The Development Team is struggling to complete work due to frequent interruptions. What should the Scrum Master do?**

- A) Allow interruptions to continue and adapt.
- B) Shield the team from interruptions and facilitate focused work.
- C) Increase the team's workload to compensate.
- D) Ignore the interruptions and focus on other issues.

**Answer:** B) Shield the team from interruptions and facilitate focused work.  
**Explanation:** The Scrum Master should protect the team from interruptions to enable them to focus on their work and deliver value. [Scrum Guide - Scrum Master Role](https://www.scrum.org/resources/scrum-guide#scrum-master-role)

### Question 29
**A Scrum Team is working on a project with high technical complexity. How should they approach Sprint Planning?**

- A) Plan the entire project in detail upfront.
- B) Focus on short-term goals and adapt as they learn more.
- C) Avoid planning and work on tasks as they come.
- D) Set fixed goals for the entire project duration.

**Answer:** B) Focus on short-term goals and adapt as they learn more.  
**Explanation:** In complex projects, the Scrum Team should focus on short-term goals and adapt their plans as they gain more information. [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)

### Question 30
**The Product Owner is frequently unavailable for important meetings. What should the Scrum Master do?**

- A) Conduct meetings without the Product Owner.
- B) Encourage the Product Owner to prioritize their availability.
- C) Assign the Product Owner's responsibilities to the Scrum Master.
- D) Skip meetings until the Product Owner is available.

**Answer:** B) Encourage the Product Owner to prioritize their availability.  
**Explanation:** The Scrum Master should ensure the Product Owner is available for important meetings to provide valuable input. [Scrum Guide - Product Owner Role](https://www.scrum.org/resources/scrum-guide#product-owner-role)

### Question 31
**A Scrum Team is working on a project with rapidly changing requirements. How should they manage the Product Backlog?**

- A) Freeze the Product Backlog to prevent changes.
- B) Continuously refine and prioritize the Product Backlog.
- C) Ignore changes and focus on the original plan.
- D) Request a fixed scope from stakeholders.

**Answer:** B) Continuously refine and prioritize the Product Backlog.  
**Explanation:** The Scrum Team should continuously refine and prioritize the Product Backlog to adapt to changing requirements. [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)

### Question 32
**The Development Team is unsure about the acceptance criteria for a Product Backlog item. What should they do?**

- A) Make assumptions and proceed with the work.
- B) Seek clarification from the Product Owner.
- C) Skip the item and move to the next one.
- D) Use previous acceptance criteria as a reference.

**Answer:** B) Seek clarification from the Product Owner.  
**Explanation:** The Development Team should clarify acceptance criteria with the Product Owner to ensure they meet the desired outcome. [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)

### Question 33
**A Scrum Team is consistently delivering increments that do not meet stakeholder expectations. What should the Scrum Master do?**

- A) Ignore stakeholder feedback and focus on the team's goals.
- B) Facilitate better communication between the team and stakeholders.
- C) Replace team members to improve performance.
- D) Extend the Sprint duration to meet expectations.

**Answer:** B) Facilitate better communication between the team and stakeholders.  
**Explanation:** The Scrum Master should ensure effective communication between the team and stakeholders to align expectations and deliver value. [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)

### Question 34
**The Product Owner is not actively participating in Sprint Planning. What should the Scrum Master do?**

- A) Conduct Sprint Planning without the Product Owner.
- B) Encourage the Product Owner to engage and provide input.
- C) Assign the Product Owner's responsibilities to the Scrum Master.
- D) Skip Sprint Planning and start the Sprint.

**Answer:** B) Encourage the Product Owner to engage and provide input.  
**Explanation:** The Scrum Master should ensure the Product Owner is actively involved in Sprint Planning to provide valuable input. [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)

### Question 35
**A Scrum Team is working on a project with multiple Scrum Teams. How should they coordinate their work?**

- A) Work independently and avoid interference.
- B) Use a shared Product Backlog and coordinate through Scrum of Scrums.
- C) Assign a project manager to oversee coordination.
- D) Merge all teams into one large team.

**Answer:** B) Use a shared Product Backlog and coordinate through Scrum of Scrums.  
**Explanation:** Coordination through a shared Product Backlog and Scrum of Scrums helps multiple teams work together effectively. [Scrum Guide - Scrum of Scrums](https://www.scrum.org/resources/scrum-guide#scrum-of-scrums)

### Question 36
**The Development Team is consistently underestimating the effort required for Product Backlog items. What should they do?**

- A) Increase all estimates by a fixed percentage.
- B) Review past estimates and adjust their estimation process.
- C) Ignore estimation errors and continue as is.
- D) Use a different estimation technique for each item.

**Answer:** B) Review past estimates and adjust their estimation process.  
**Explanation:** The Development Team should learn from past estimations and improve their estimation process for future items. [Scrum Guide - Estimation](https://www.scrum.org/resources/scrum-guide#estimation)

### Question 37
**A Scrum Team is facing resistance from stakeholders who are unfamiliar with Scrum. What should the Scrum Master do?**

- A) Ignore stakeholder concerns and focus on the team's work.
- B) Educate stakeholders about Scrum and its benefits.
- C) Replace stakeholders with those familiar with Scrum.
- D) Adjust the Scrum process to meet stakeholder demands.

**Answer:** B) Educate stakeholders about Scrum and its benefits.  
**Explanation:** The Scrum Master should educate stakeholders about Scrum to gain their support and understanding. [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)

### Question 38
**The Product Owner wants to add a new feature to the current Sprint. How should the Scrum Team handle this request?**

- A) Implement the feature immediately.
- B) Discuss the request with the Development Team and adjust the Sprint Backlog if necessary.
- C) Reject the request to maintain focus.
- D) Add the feature to the next Sprint without discussion.

**Answer:** B) Discuss the request with the Development Team and adjust the Sprint Backlog if necessary.  
**Explanation:** The Scrum Team should collaborate to assess the impact of the new feature and adjust the Sprint Backlog if needed. [Scrum Guide - Sprint Backlog](https://www.scrum.org/resources/scrum-guide#sprint-backlog)

### Question 39
**A Scrum Team is working on a project with a fixed deadline. How should they manage their work to meet the deadline?**

- A) Work overtime to meet the deadline.
- B) Focus on delivering the highest priority items first.
- C) Extend the deadline to ensure quality.
- D) Reduce the scope of the project.

**Answer:** B) Focus on delivering the highest priority items first.  
**Explanation:** The Scrum Team should prioritize high-value items to ensure they deliver the most important features by the deadline. [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)

### Question 40
**The Development Team is struggling to complete work due to frequent interruptions. What should the Scrum Master do?**

- A) Allow interruptions to continue and adapt.
- B) Shield the team from interruptions and facilitate focused work.
- C) Increase the team's workload to compensate.
- D) Ignore the interruptions and focus on other issues.

**Answer:** B) Shield the team from interruptions and facilitate focused work.  
**Explanation:** The Scrum Master should protect the team from interruptions to enable them to focus on their work and deliver value. [Scrum Guide - Scrum Master Role](https://www.scrum.org/resources/scrum-guide#scrum-master-role)

### Question 41
**A Scrum Team is working on a project with high technical complexity. How should they approach Sprint Planning?**

- A) Plan the entire project in detail upfront.
- B) Focus on short-term goals and adapt as they learn more.
- C) Avoid planning and work on tasks as they come.
- D) Set fixed goals for the entire project duration.

**Answer:** B) Focus on short-term goals and adapt as they learn more.  
**Explanation:** In complex projects, the Scrum Team should focus on short-term goals and adapt their plans as they gain more information. [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)

### Question 42
**The Product Owner is frequently unavailable for important meetings. What should the Scrum Master do?**

- A) Conduct meetings without the Product Owner.
- B) Encourage the Product Owner to prioritize their availability.
- C) Assign the Product Owner's responsibilities to the Scrum Master.
- D) Skip meetings until the Product Owner is available.

**Answer:** B) Encourage the Product Owner to prioritize their availability.  
**Explanation:** The Scrum Master should ensure the Product Owner is available for important meetings to provide valuable input. [Scrum Guide - Product Owner Role](https://www.scrum.org/resources/scrum-guide#product-owner-role)

### Question 43
**A Scrum Team is working on a project with rapidly changing requirements. How should they manage the Product Backlog?**

- A) Freeze the Product Backlog to prevent changes.
- B) Continuously refine and prioritize the Product Backlog.
- C) Ignore changes and focus on the original plan.
- D) Request a fixed scope from stakeholders.

**Answer:** B) Continuously refine and prioritize the Product Backlog.  
**Explanation:** The Scrum Team should continuously refine and prioritize the Product Backlog to adapt to changing requirements. [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)

### Question 44
**The Development Team is unsure about the acceptance criteria for a Product Backlog item. What should they do?**

- A) Make assumptions and proceed with the work.
- B) Seek clarification from the Product Owner.
- C) Skip the item and move to the next one.
- D) Use previous acceptance criteria as a reference.

**Answer:** B) Seek clarification from the Product Owner.  
**Explanation:** The Development Team should clarify acceptance criteria with the Product Owner to ensure they meet the desired outcome. [Scrum Guide - Product Backlog](https://www.scrum.org/resources/scrum-guide#product-backlog)

### Question 45
**A Scrum Team is consistently delivering increments that do not meet stakeholder expectations. What should the Scrum Master do?**

- A) Ignore stakeholder feedback and focus on the team's goals.
- B) Facilitate better communication between the team and stakeholders.
- C) Replace team members to improve performance.
- D) Extend the Sprint duration to meet expectations.

**Answer:** B) Facilitate better communication between the team and stakeholders.  
**Explanation:** The Scrum Master should ensure effective communication between the team and stakeholders to align expectations and deliver value. [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)

### Question 46
**The Product Owner is not actively participating in Sprint Planning. What should the Scrum Master do?**

- A) Conduct Sprint Planning without the Product Owner.
- B) Encourage the Product Owner to engage and provide input.
- C) Assign the Product Owner's responsibilities to the Scrum Master.
- D) Skip Sprint Planning and start the Sprint.

**Answer:** B) Encourage the Product Owner to engage and provide input.  
**Explanation:** The Scrum Master should ensure the Product Owner is actively involved in Sprint Planning to provide valuable input. [Scrum Guide - Sprint Planning](https://www.scrum.org/resources/scrum-guide#sprint-planning)

### Question 47
**A Scrum Team is working on a project with multiple Scrum Teams. How should they coordinate their work?**

- A) Work independently and avoid interference.
- B) Use a shared Product Backlog and coordinate through Scrum of Scrums.
- C) Assign a project manager to oversee coordination.
- D) Merge all teams into one large team.

**Answer:** B) Use a shared Product Backlog and coordinate through Scrum of Scrums.  
**Explanation:** Coordination through a shared Product Backlog and Scrum of Scrums helps multiple teams work together effectively. [Scrum Guide - Scrum of Scrums](https://www.scrum.org/resources/scrum-guide#scrum-of-scrums)

### Question 48
**The Development Team is consistently underestimating the effort required for Product Backlog items. What should they do?**

- A) Increase all estimates by a fixed percentage.
- B) Review past estimates and adjust their estimation process.
- C) Ignore estimation errors and continue as is.
- D) Use a different estimation technique for each item.

**Answer:** B) Review past estimates and adjust their estimation process.  
**Explanation:** The Development Team should learn from past estimations and improve their estimation process for future items. [Scrum Guide - Estimation](https://www.scrum.org/resources/scrum-guide#estimation)

### Question 49
**A Scrum Team is facing resistance from stakeholders who are unfamiliar with Scrum. What should the Scrum Master do?**

- A) Ignore stakeholder concerns and focus on the team's work.
- B) Educate stakeholders about Scrum and its benefits.
- C) Replace stakeholders with those familiar with Scrum.
- D) Adjust the Scrum process to meet stakeholder demands.

**Answer:** B) Educate stakeholders about Scrum and its benefits.  
**Explanation:** The Scrum Master should educate stakeholders about Scrum to gain their support and understanding. [Scrum Guide - Stakeholder Engagement](https://www.scrum.org/resources/scrum-guide#stakeholder-engagement)

### Question 50
**The Product Owner wants to add a new feature to the current Sprint. How should the Scrum Team handle this request?**

- A) Implement the feature immediately.
- B) Discuss the request with the Development Team and adjust the Sprint Backlog if necessary.
- C) Reject the request to maintain focus.
- D) Add the feature to the next Sprint without discussion.

**Answer:** B) Discuss the request with the Development Team and adjust the Sprint Backlog if necessary.  
**Explanation:** The Scrum Team should collaborate to assess the impact of the new feature and adjust the Sprint Backlog if needed. [Scrum Guide - Sprint Backlog](https://www.scrum.org/resources/scrum-guide#sprint-backlog)

---

This mock exam is designed to provide a comprehensive practice experience for the Professional Scrum Master I certification. Each question is crafted to reflect the style and difficulty level of the real exam, ensuring that you are well-prepared for the certification assessment.